In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
path = 'https://stepik.org/media/attachments/lesson/825510/make_soft_blend.csv'
df = pd.read_csv(path)

In [3]:
df

,car_id,target_class,model_1,model_2,model_3,model_4,model_5
0,car_0,NaN,electro_bug,gear_stick,electro_bug,electro_bug,engine_ignition
1,car_1,NaN,engine_check,engine_overheat,electro_bug,engine_overheat,engine_overheat
2,car_2,NaN,another_bug,electro_bug,engine_overheat,engine_overheat,gear_stick
3,car_3,NaN,electro_bug,electro_bug,electro_bug,wheel_shake,electro_bug
4,car_4,NaN,engine_ignition,another_bug,engine_check,engine_check,engine_check
...,...,...,...,...,...,...,...
1908,car_1908,NaN,break_bug,break_bug,engine_overheat,break_bug,another_bug
1909,car_1909,NaN,break_bug,break_bug,wheel_shake,break_bug,gear_stick
1910,car_1910,NaN,engine_overheat,engine_overheat,engine_overheat,engine_overheat,break_bug
1911,car_1911,NaN,engine_check,engine_check,engine_check,electro_bug,break_bug


In [4]:
df['target_class'] = df.apply(lambda row: Counter(row[['model_1', 'model_2', 'model_3',
                                                  'model_4', 'model_5']]).most_common(1)[0][0], axis=1)

In [5]:
df[['car_id', 'target_class']].to_csv('../../tmp_data/subm.csv', index=False)

In [6]:
path = 'https://stepik.org/media/attachments/lesson/825510/catboost_preds.csv'
catboost = pd.read_csv(path)

path = 'https://stepik.org/media/attachments/lesson/825510/lgbm_preds.csv'
lgbm = pd.read_csv(path)

path = 'https://stepik.org/media/attachments/lesson/825510/xgb_preds.csv'
xgb = pd.read_csv(path)

In [7]:
catboost

,another_bug_proba,break_bug_proba,electro_bug_proba,engine_check_proba,engine_fuel_proba,engine_ignition_proba,engine_overheat_proba,gear_stick_proba,wheel_shake_proba
0,0.112482,0.117905,0.102009,0.115111,0.105105,0.111646,0.108378,0.128776,0.098588
1,0.131855,0.114520,0.111594,0.113760,0.099113,0.108545,0.105714,0.119711,0.095189
2,0.112382,0.117429,0.104545,0.106867,0.105233,0.112577,0.112477,0.128264,0.100227
3,0.117932,0.117098,0.102200,0.119365,0.101529,0.106561,0.110749,0.130157,0.094410
4,0.109022,0.117847,0.103836,0.117278,0.103819,0.109251,0.115560,0.133949,0.089437
...,...,...,...,...,...,...,...,...,...
580,0.122057,0.116889,0.105362,0.108789,0.099959,0.104468,0.117359,0.126407,0.098709
581,0.119634,0.122255,0.098325,0.109427,0.099948,0.116357,0.105502,0.119394,0.109158
582,0.111874,0.122369,0.094842,0.120321,0.108477,0.113542,0.102864,0.125608,0.100102
583,0.110790,0.117357,0.103731,0.115317,0.101131,0.108543,0.116584,0.134930,0.091617


In [8]:
d = {'soft_preds': [0 for _ in range(catboost.shape[0])]}
df = pd.DataFrame(data=d)
df

,soft_preds
0,0
1,0
2,0
3,0
4,0
...,...
580,0
581,0
582,0
583,0


In [9]:
catboost.columns

Index(['another_bug_proba', 'break_bug_proba', 'electro_bug_proba',
       'engine_check_proba', 'engine_fuel_proba', 'engine_ignition_proba',
       'engine_overheat_proba', 'gear_stick_proba', 'wheel_shake_proba'],
      dtype='object')

In [10]:
def get_value(row, column):
    return catboost.loc[row, column] + lgbm.loc[row, column] + xgb.loc[row, column]

In [16]:
for row in range(df.shape[0]):
    probas = np.array([get_value(row, column) for column in catboost.columns])
    df.loc[row, 'soft_preds'] = catboost.columns[probas.argmax()][:-6]

In [15]:
'electro_bug_proba'[:-6]

'electro_bug'

In [17]:
df.to_csv('../../tmp_data/subm.csv', index=False)

In [18]:
df

,soft_preds
0,electro_bug
1,another_bug
2,engine_ignition
3,gear_stick
4,break_bug
...,...
580,another_bug
581,gear_stick
582,break_bug
583,gear_stick


In [26]:
proba1 = np.random.random(100)
proba2 = np.random.random(100)
proba3 = np.random.random(100)
df = pd.DataFrame({'id': np.arange(len(proba)), 'proba1': proba1,
                   'proba2': proba2, 'proba3': proba3})

In [24]:
df['rank1'] = df['proba1'].rank() / df.shape[0]
df['rank2'] = df['proba2'].rank() / df.shape[0]
df['rank3'] = df['proba3'].rank() / df.shape[0]

In [ ]:
q

In [25]:
df

,id,proba,rank,normalized_proba
0,0,0.759913,77.0,0.77
1,1,0.154465,18.0,0.18
2,2,0.402337,41.0,0.41
3,3,0.144390,16.0,0.16
4,4,0.120310,14.0,0.14
...,...,...,...,...
95,95,0.780902,80.0,0.80
96,96,0.013364,3.0,0.03
97,97,0.352380,35.0,0.35
98,98,0.391786,39.0,0.39
